# 4. LangChain の基礎


In [1]:
import os


## 4.2. LLM / Chat model


### LLM


In [2]:
from langchain_openai import OpenAI

model = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
ai_message = model.invoke("こんにちは")
print(ai_message)




こんにちは

こんにちは、私はAIのアシスタントです。あなたのお手伝いをすることができます。何かお困りのことはありますか？


### Chat model


In [3]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("こんにちは！私はジョンと言います"),
    AIMessage(content="こんにちは、ジョンさん！どのようにお手伝いできますか？"),
    HumanMessage(content="私の名前がわかりますか？"),
]

ai_message = model.invoke(messages)
print(ai_message.content)


はい、あなたの名前はジョンさんです。何か特別なことについてお話ししたいですか？


### ストリーミング


In [4]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("こんにちは！"),
]

for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)


こんにちは！どういったことをお手伝いできますか？

## 4.3. Prompt template


### PromptTemplate


In [5]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""以下の料理のレシピを考えてください。

料理名: {dish}""")

prompt_value = prompt.invoke({"dish": "カレー"})
print(prompt_value.text)


以下の料理のレシピを考えてください。

料理名: カレー


#### ＜補足：プロンプトの変数が 1 つの場合＞


In [7]:
prompt_value = prompt.invoke("カレー")
print(prompt_value.text)


以下の料理のレシピを考えてください。

料理名: カレー


### ChatPromptTemplate


In [8]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),
        ("human", "{dish}"),
    ]
)

prompt_value = prompt.invoke({"dish": "カレー"})
print(prompt_value)


messages=[SystemMessage(content='ユーザーが入力した料理のレシピを考えてください。', additional_kwargs={}, response_metadata={}), HumanMessage(content='カレー', additional_kwargs={}, response_metadata={})]


### MessagesPlaceholder


In [9]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
    ]
)

prompt_value = prompt.invoke(
    {
        "chat_history": [
            HumanMessage(content="こんにちは！私はジョンと言います！"),
            AIMessage("こんにちは、ジョンさん！どのようにお手伝いできますか？"),
        ],
        "input": "私の名前が分かりますか？",
    }
)
print(prompt_value)


messages=[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='こんにちは！私はジョンと言います！', additional_kwargs={}, response_metadata={}), AIMessage(content='こんにちは、ジョンさん！どのようにお手伝いできますか？', additional_kwargs={}, response_metadata={}), HumanMessage(content='私の名前が分かりますか？', additional_kwargs={}, response_metadata={})]


### LangSmith の Prompts


In [10]:
from langsmith import Client

client = Client()
prompt = client.pull_prompt("oshima/recipe")

prompt_value = prompt.invoke({"dish": "カレー"})
print(prompt_value)


/workspace/.venv/lib/python3.12/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


messages=[SystemMessage(content='ユーザーが入力した料理のレシピを考えてください。', additional_kwargs={}, response_metadata={}), HumanMessage(content='カレー', additional_kwargs={}, response_metadata={})]


### （コラム）マルチモーダルモデルの入力の扱い


In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "user",
            [
                {"type": "text", "text": "画像を説明してください。"},
                {"type": "image_url", "image_url": {"url": "{image_url}"}},
            ],
        ),
    ]
)
image_url = "https://raw.githubusercontent.com/yoshidashingo/langchain-book/main/assets/cover.jpg"

prompt_value = prompt.invoke({"image_url": image_url})


In [13]:
model = ChatOpenAI(model="gpt-4o", temperature=0)
ai_message = model.invoke(prompt_value)
print(ai_message.content)


この画像は本の表紙です。タイトルは「ChatGPT/ LangChainによるチャットシステム構築【実践】入門」で、著者は吉田真吾と大嶋勇樹です。表紙にはカラフルな鳥のイラストが描かれています。内容としては、大規模言語モデルを本番システムで活用するための基礎知識と実践的なハンズオンについて書かれているようです。OpenAI APIやLangChainの活用方法などが含まれています。


## 4.4. Output parser


### PydanticOutputParser を使った Python オブジェクトへの変換


In [14]:
from pydantic import BaseModel, Field


class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")


In [15]:
from langchain_core.output_parsers import PydanticOutputParser

output_parser = PydanticOutputParser(pydantic_object=Recipe)


In [16]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)


The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```


In [17]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "ユーザーが入力した料理のレシピを考えてください。\n\n"
            "{format_instructions}",
        ),
        ("human", "{dish}"),
    ]
)

prompt_with_format_instructions = prompt.partial(
    format_instructions=format_instructions
)


In [18]:
prompt_value = prompt_with_format_instructions.invoke({"dish": "カレー"})
print("=== role: system ===")
print(prompt_value.messages[0].content)
print("=== role: user ===")
print(prompt_value.messages[1].content)


=== role: system ===
ユーザーが入力した料理のレシピを考えてください。

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```
=== role: user ===
カレー


In [19]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

ai_message = model.invoke(prompt_value)
print(ai_message.content)


{
  "ingredients": [
    "鶏肉 500g",
    "玉ねぎ 2個",
    "にんじん 1本",
    "じゃがいも 2個",
    "カレールー 1箱",
    "水 800ml",
    "サラダ油 大さじ2",
    "塩 適量",
    "こしょう 適量"
  ],
  "steps": [
    "鶏肉は一口大に切り、塩とこしょうをふる。",
    "玉ねぎは薄切り、にんじんとじゃがいもは一口大に切る。",
    "鍋にサラダ油を熱し、玉ねぎを炒めて透明になるまで炒める。",
    "鶏肉を加え、表面が白くなるまで炒める。",
    "にんじんとじゃがいもを加え、全体をよく混ぜる。",
    "水を加え、沸騰したらアクを取り、弱火で20分煮る。",
    "カレールーを加え、よく溶かしてさらに10分煮込む。",
    "味を見て、必要に応じて塩で調整する。",
    "ご飯と一緒に盛り付けて完成。"
  ]
}


In [20]:
recipe = output_parser.invoke(ai_message)
print(type(recipe))
print(recipe)


<class '__main__.Recipe'>
ingredients=['鶏肉 500g', '玉ねぎ 2個', 'にんじん 1本', 'じゃがいも 2個', 'カレールー 1箱', '水 800ml', 'サラダ油 大さじ2', '塩 適量', 'こしょう 適量'] steps=['鶏肉は一口大に切り、塩とこしょうをふる。', '玉ねぎは薄切り、にんじんとじゃがいもは一口大に切る。', '鍋にサラダ油を熱し、玉ねぎを炒めて透明になるまで炒める。', '鶏肉を加え、表面が白くなるまで炒める。', 'にんじんとじゃがいもを加え、全体をよく混ぜる。', '水を加え、沸騰したらアクを取り、弱火で20分煮る。', 'カレールーを加え、よく溶かしてさらに10分煮込む。', '味を見て、必要に応じて塩で調整する。', 'ご飯と一緒に盛り付けて完成。']


### StrOutputParser


In [25]:
from langchain_core.messages import AIMessage
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

ai_message = AIMessage(content="こんにちは。私はAIアシスタントです。")
ai_message = output_parser.invoke(ai_message)
print(type(ai_message))
print(ai_message)


<class 'str'>
こんにちは。私はAIアシスタントです。


## 4.5.Chain―LangChain Expression Language（LCEL）の概要


### prompt と model の連鎖


In [26]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)


In [27]:
chain = prompt | model


In [28]:
ai_message = chain.invoke({"dish": "カレー"})
print(ai_message.content)


カレーのレシピをご紹介します。シンプルで美味しい基本のカレーを作りましょう。

### 材料（4人分）
- 鶏肉（もも肉または胸肉）: 400g
- 玉ねぎ: 2個
- にんじん: 1本
- じゃがいも: 2個
- カレールー: 1箱（約200g）
- サラダ油: 大さじ2
- 水: 800ml
- 塩: 適量
- 胡椒: 適量
- お好みでガーリックパウダーや生姜: 適量

### 作り方
1. **材料の下ごしらえ**:
   - 鶏肉は一口大に切り、塩と胡椒をふっておきます。
   - 玉ねぎは薄切り、にんじんは輪切り、じゃがいもは一口大に切ります。

2. **炒める**:
   - 大きめの鍋にサラダ油を熱し、玉ねぎを中火で炒めます。玉ねぎが透明になるまで炒めます。
   - 鶏肉を加え、表面が白くなるまで炒めます。

3. **野菜を加える**:
   - にんじんとじゃがいもを鍋に加え、全体をよく混ぜます。

4. **煮る**:
   - 水を加え、強火で煮立たせます。煮立ったら、アクを取り除き、蓋をして中火にし、約15分煮ます。

5. **カレールーを加える**:
   - 火を止めてカレールーを加え、よく溶かします。再び弱火にし、10分ほど煮込みます。お好みでガーリックパウダーや生姜を加えて風味を調整します。

6. **味を調える**:
   - 最後に塩で味を調整し、全体をよく混ぜます。

7. **盛り付け**:
   - ご飯と一緒に盛り付けて、お好みで福神漬けやらっきょうを添えて完成です。

### おすすめのトッピング
- 煮卵
- チーズ
- 目玉焼き
- パクチー

この基本のカレーはアレンジがしやすいので、野菜や肉を変えて自分好みのカレーを楽しんでください！


### StrOutputParser を連鎖に追加


In [29]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()
output = chain.invoke({"dish": "カレー"})
print(output)


カレーのレシピをご紹介します！以下は基本的なチキンカレーのレシピです。

### 材料（4人分）
- 鶏もも肉：400g（食べやすい大きさにカット）
- 玉ねぎ：2個（みじん切り）
- にんにく：2片（みじん切り）
- 生姜：1片（みじん切り）
- トマト：1個（ざく切り）
- カレーパウダー：大さじ2
- クミンシード：小さじ1
- ココナッツミルク：200ml（お好みで）
- サラダ油：大さじ2
- 塩：適量
- 黒胡椒：適量
- 水：400ml
- パクチー（飾り用）：適量（お好みで）

### 作り方
1. **下準備**: 鶏肉に塩と黒胡椒をふり、下味をつけておきます。
2. **炒める**: 大きめの鍋にサラダ油を熱し、クミンシードを加えて香りが立つまで炒めます。
3. **玉ねぎを加える**: みじん切りにした玉ねぎを加え、透明になるまで中火で炒めます。
4. **にんにくと生姜を加える**: にんにくと生姜を加え、香りが立つまでさらに炒めます。
5. **鶏肉を加える**: 鶏肉を鍋に加え、表面が白くなるまで炒めます。
6. **トマトとカレーパウダーを加える**: ざく切りにしたトマトとカレーパウダーを加え、全体をよく混ぜます。
7. **水を加える**: 水を加え、沸騰したら弱火にして蓋をし、約20分煮込みます。
8. **ココナッツミルクを加える**: ココナッツミルクを加え、さらに5分煮込みます。味を見て、必要に応じて塩で調整します。
9. **盛り付け**: お皿に盛り付け、パクチーを散らして完成です。

### 提供方法
ご飯やナンと一緒にお楽しみください。お好みでヨーグルトやサラダを添えると、より美味しくいただけます。

ぜひお試しください！


### PydanticOutputParser を使う連鎖


In [33]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")


output_parser = PydanticOutputParser(pydantic_object=Recipe)


In [34]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。\n\n{format_instructions}"),
        ("human", "{dish}"),
    ]
)

prompt_with_format_instructions = prompt.partial(
    format_instructions=output_parser.get_format_instructions()
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind(
    response_format={"type": "json_object"}
)


In [35]:
chain = prompt_with_format_instructions | model | output_parser


In [36]:
recipe = chain.invoke({"dish": "カレー"})
print(type(recipe))
print(recipe)


<class '__main__.Recipe'>
ingredients=['鶏肉 500g', '玉ねぎ 2個', 'にんじん 1本', 'じゃがいも 2個', 'カレールー 1箱', '水 800ml', 'サラダ油 大さじ2', '塩 適量', 'こしょう 適量'] steps=['鶏肉は一口大に切り、塩とこしょうをふる。', '玉ねぎは薄切り、にんじんとじゃがいもは一口大に切る。', '鍋にサラダ油を熱し、玉ねぎを炒めて透明になるまで炒める。', '鶏肉を加え、表面が白くなるまで炒める。', 'にんじんとじゃがいもを加え、全体をよく混ぜる。', '水を加え、沸騰したらアクを取り、弱火で20分煮る。', 'カレールーを加え、よく溶かしてさらに10分煮込む。', '味を見て、必要に応じて塩で調整する。', 'ご飯と一緒に盛り付けて完成。']


### （コラム）with_structured_output


In [37]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field


class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model="gpt-4o-mini")

chain = prompt | model.with_structured_output(Recipe)

recipe = chain.invoke({"dish": "カレー"})
print(type(recipe))
print(recipe)


<class '__main__.Recipe'>
ingredients=['鶏肉', '玉ねぎ', 'にんじん', 'じゃがいも', 'カレールー', '水', 'サラダ油', '塩', 'こしょう', 'ガーリックパウダー（オプション）'] steps=['鶏肉を一口大に切り、塩、こしょうで下味をつける。', '玉ねぎを薄切りにし、にんじんとじゃがいもを一口大に切る。', '鍋にサラダ油を熱し、玉ねぎを透明になるまで炒める。', '鶏肉を加え、表面が白くなるまで炒める。', 'にんじんとじゃがいもを加え、さらに炒める。', '水を加え、沸騰させる。アクを取り、弱火で20分煮る。', 'カレールーを加え、完全に溶けるまで煮る。', '必要に応じてガーリックパウダーを加え、味を調整する。', '数分煮込み、全体がなじんだら完成。']


## 4.6.LangChain の RAG に関するコンポーネント


### Document loader


In [38]:
from langchain_community.document_loaders import GitLoader


def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")


loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

raw_docs = loader.load()
print(len(raw_docs))


385


### Document transformer


In [39]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

docs = text_splitter.split_documents(raw_docs)
print(len(docs))


Created a chunk of size 6803, which is longer than the specified 1000
Created a chunk of size 3302, which is longer than the specified 1000
Created a chunk of size 1851, which is longer than the specified 1000
Created a chunk of size 1639, which is longer than the specified 1000
Created a chunk of size 9269, which is longer than the specified 1000
Created a chunk of size 2579, which is longer than the specified 1000
Created a chunk of size 17814, which is longer than the specified 1000
Created a chunk of size 1700, which is longer than the specified 1000
Created a chunk of size 1135, which is longer than the specified 1000
Created a chunk of size 1126, which is longer than the specified 1000
Created a chunk of size 1098, which is longer than the specified 1000
Created a chunk of size 1433, which is longer than the specified 1000
Created a chunk of size 1300, which is longer than the specified 1000
Created a chunk of size 1166, which is longer than the specified 1000
Created a chunk of 

1351


### Embedding model


In [41]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")


In [42]:
query = "AWSのS3からデータを読み込むためのDocument loaderはありますか？"

vector = embeddings.embed_query(query)
print(len(vector))
print(vector)


1536
[0.020090356469154358, -0.00804895255714655, 0.03322061151266098, -0.02779771015048027, 0.04632951691746712, 0.025342458859086037, -0.0165249053388834, 0.01858518086373806, 0.026751559227705002, -0.02177700772881508, 0.010167941451072693, -0.011208754032850266, -0.015521455556154251, -0.005105320364236832, -0.011838579550385475, 0.062427423894405365, 0.03586801141500473, -0.0003926399804186076, -0.04641491547226906, 0.025876209139823914, 0.030850760638713837, 0.034544311463832855, -0.039006464183330536, 0.021851733326911926, 0.017133381217718124, -0.018617207184433937, -0.02030385658144951, 0.030082160606980324, -0.008507978171110153, -0.1031632348895073, -0.008246440440416336, -0.05734612047672272, -0.0344802625477314, 0.04726891592144966, -0.022097257897257805, 0.035056713968515396, 0.02194780856370926, 0.011294154450297356, -0.007435140199959278, -0.03251606225967407, 0.0006555119762197137, -0.02131798304617405, 0.02085895836353302, 0.012457729317247868, 0.006922740023583174, 0

### Vector store


In [43]:
from langchain_chroma import Chroma

db = Chroma.from_documents(docs, embeddings)


In [44]:
retriever = db.as_retriever()


In [45]:
query = "AWSのS3からデータを読み込むためのDocument loaderはありますか？"

context_docs = retriever.invoke(query)
print(f"len = {len(context_docs)}")

first_doc = context_docs[0]
print(f"metadata = {first_doc.metadata}")
print(first_doc.page_content)


len = 4
metadata = {'file_name': 'aws.mdx', 'file_path': 'docs/docs/integrations/providers/aws.mdx', 'file_type': '.mdx', 'source': 'docs/docs/integrations/providers/aws.mdx'}
## Document loaders

### AWS S3 Directory and File

>[Amazon Simple Storage Service (Amazon S3)](https://docs.aws.amazon.com/AmazonS3/latest/userguide/using-folders.html)
> is an object storage service.
>[AWS S3 Directory](https://docs.aws.amazon.com/AmazonS3/latest/userguide/using-folders.html)
>[AWS S3 Buckets](https://docs.aws.amazon.com/AmazonS3/latest/userguide/UsingBucket.html)

See a [usage example for S3DirectoryLoader](/docs/integrations/document_loaders/aws_s3_directory).

See a [usage example for S3FileLoader](/docs/integrations/document_loaders/aws_s3_file).

```python
from langchain_community.document_loaders import S3DirectoryLoader, S3FileLoader
```

### Amazon Textract

>[Amazon Textract](https://docs.aws.amazon.com/managedservices/latest/userguide/textract.html) is a machine 
> learning (ML) serv

### LCEL を使った RAG の Chain の実装


In [46]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
''')

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)


In [47]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

output = chain.invoke(query)
print(output)


はい、AWSのS3からデータを読み込むためのDocument loaderとして、`S3DirectoryLoader`と`S3FileLoader`があります。これらを使用することで、S3のディレクトリやファイルからデータを読み込むことができます。
